In [6]:

import numpy as np

from landlab import RasterModelGrid

def fill(Z):
    Ni = Z.shape[0]
    Nj = Z.shape[1]
    border = np.zeros((Ni, Nj)).astype(np.int8)
    border[:, 0] = 1
    border[:, -1] = 1
    border[0, :] = 1
    border[-1, :] = 1
    W = np.where(border == 0, np.max(Z) + 0.01, Z)
    eps = 0.0000001
    smt_done = 1
    while smt_done == 1:
        smt_done = 0
        proc_ext = np.where((W > Z) & (border == 0), 1, 0).astype(np.int8)
        list_nb = neighbour_list(W, -1)
        for nb in range(0, 8):
            case_1 = np.where((proc_ext == 1) & (Z >= list_nb[nb] + eps), 1, 0).astype(np.int8)
            case_2 = np.where((proc_ext == 1) & (case_1 == 0) & (W > list_nb[nb] + eps), 1, 0).astype(np.int8)
            Wnew = np.where(case_1 == 1, Z, W)
            Wnew = np.where(case_2 == 1, list_nb[nb] + eps, Wnew)
            if np.sum(np.abs(W - Wnew)) > 0:
                smt_done = 1
            W = np.copy(Wnew)
            list_nb = neighbour_list(W, -1)
    return W

def neighbour_list(A, border_value):
    Ni = A.shape[0]
    Nj = A.shape[1]

    A_7 = np.append(border_value * np.ones((1, Nj)), A[0: Ni - 1, :].reshape(Ni - 1, Nj), axis=0)
    A_5 = np.append(border_value * np.ones((Ni, 1)), A[:, 0: Nj - 1].reshape(Ni, Nj - 1), axis=1)
    A_1 = np.append(A[:, 1: Nj].reshape(Ni, Nj - 1), border_value * np.ones((Ni, 1)), axis=1)
    A_3 = np.append(A[1: Ni, :].reshape(Ni - 1, Nj), border_value * np.ones((1, Nj)), axis=0)

    A_6 = np.append(border_value * np.ones((Ni, 1)), A_7[:, 0: Nj - 1].reshape(Ni, Nj - 1), axis=1)
    A_8 = np.append(A_7[:, 1: Nj].reshape(Ni, Nj - 1), border_value * np.ones((Ni, 1)), axis=1)
    A_4 = np.append(border_value * np.ones((Ni, 1)), A_3[:, 0: Nj - 1].reshape(Ni, Nj - 1), axis=1)
    A_2 = np.append(A_3[:, 1: Nj].reshape(Ni, Nj - 1), border_value * np.ones((Ni, 1)), axis=1)

    list_neighbour = []

    list_neighbour.append(A_1)
    list_neighbour.append(A_2)
    list_neighbour.append(A_3)
    list_neighbour.append(A_4)
    list_neighbour.append(A_5)
    list_neighbour.append(A_6)
    list_neighbour.append(A_7)
    list_neighbour.append(A_8)

    return list_neighbour


nrows= 300
ncols=300
dx =5



mu, sigma         = 0, 0.00001
initial_roughness = np.random.normal(mu, sigma, (nrows,ncols))
z  = np.zeros((nrows,ncols))
z +=       initial_roughness 
z  =                 fill(z)
z1 =   z + np.abs(np.min(z))
#print(np.min(z1),np.max(z1))
z1[:,0]                  = 0
z1[:,ncols-1]            = 0
z1[nrows-1,:]            = 0
z1[0,:]                  = 0

z =z1
#z = z.astype(np.float64)
mg     =                           RasterModelGrid((nrows,ncols), dx)
mg.at_node['topographic__elevation']   =     z.reshape(nrows * ncols)

for edge in (mg.nodes_at_left_edge, mg.nodes_at_right_edge):
    mg.status_at_node[edge] = RasterModelGrid.BC_NODE_IS_FIXED_VALUE
for edge in (mg.nodes_at_bottom_edge,mg.nodes_at_top_edge):
    mg.status_at_node[edge] = RasterModelGrid.BC_NODE_IS_CLOSED